# Lecture 2 - model queries

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The activities performed are:
- how do we access the data from other machines?
    - logical access: not all queries make sense, and we only want to present a reasonable interface
        - `Course 1 - N CourseLecture N - 1 Lecture` in the DB might be published as `Course 1 - N Lecture`
        - Unpublished/draft lectures might be filtered (and thus not given as result)
    - security: not all data might be seen by everyone; we only want to allow users to see/write data they are allowed to see/write:
        - suppose students are grouped in classes: `Class 1 - N Student`
        - courses are bound to classes: `Course 1 - N Class`
        - a course also has exams: `Course 1 - N Exam`
        - suppose a student asks for all available exams: which exams should he get?
            - _all exams of all courses bound to the class of the student_
            - otherwise students get to see illegally available information, and you do not want to be the programmer to explain this to your manager :)
- this means that we need a way to perform `queries` when retrieving data from our ORM
- the ORM so far was just a series of data types
- the ORM also presents an interface to access the tables, in EF called `database context`
    - each table gets a property which has the same name as the table, and as type a generic container (`DbSet<T>`) with the class mapping the table type `public DbSet<Teacher> Teacher { get; set; }`
- there is an issue though: suppose we want to perform a query on a huge table, with gigabytes of data: how will this happen?
    - should we copy all the table data in memory, and then query it with `while`/`foreach` loops?
    - do we expect to discard most of the data?
    - is this feasible for performance? What if the database is stored on another machine?
- we want to be able to run queries *as if* we were running them locally, but instead accumulating the instructions to later execute, in one go, in the database; this way, we only request the result once, when it is actually used:
    - build a `IQueryable<T>` object, which will contain the data needed to generate an SQL query which will yield a series of `T`'s
        - use and combine other queries to get to the desired `IQueryable<T>`
    - send our query to the database and get the results
        - this is done by invoking `ToList` on the `IQueryable<T>`, leading to a `List<T>`
    - the `IQueryable<T>` contains no data, and is "owned" by the database
    - the `List<T>` contains the data, and is "owned" by the application
- how do we bootstrap a query?
    - the type `DbSet<T>` encapsulates a table access, and is also an `IQueryable<T>`, so this will usually be our starting point for a query
- how do we transform queries?
    - we can map them (queries are functors): `Select: IQueryable<T>, Func<T,U> => IQueryable<U>`
        - example: `dbContext.Students.Select(s => s.Name)` is an `IQueryable<string>`
        - example: `dbContext.Students.Select(s => s.Name).Select(n => n.Length)` is an `IQueryable<int>`
        - ...
    - we can filter them (queries are functors): `Where: IQueryable<T>, Func<T,bool> => IQueryable<T>`
        - example: `dbContext.Students.Where(s => s.Name != "Jim")` is an `IQueryable<Student>`
        - example: `dbContext.Course.Where(s => s.StudyPoints > 2)` is an `IQueryable<Course>`
        - ...
    - we can also perform data aggregation: `Any: IQueryable<T>, Func<T,bool> => bool`
        - example: `dbContext.Students.Any(s => s.Name == "Pim")`
        - example: `dbContext.Courses.Any(c => c.StartDate >= DateTime.Now && c.EndDate <= DateTime.Now)`
        - ...
    - there are various other aggregation functions: 
        - `Count: IQueryable<T> => int`
        - `Count: IQueryable<T>, Func<T,bool> => int`
        - `Aggregate: IQueryable<T>, U, Func<U,T,U> => IQueryable<U>`
            - example: `dbContext.Students.Aggregate("", (names,s) => names + " " + s)`
            - ...
            - Note that aggregate is actually sufficient to implement all other aggregation operators
                - show how to build `Count` and `Any` from `Aggregate`
        - And many more operators, use GIYF ("Google Is Your Friend") for them
    - there also exists a fundamental mechanism to **bind** queries together; suppose you had:
        - an `IQueryable<T>`, which will yield a series of values of type `T`
        - a transformation function `Func<T, IQueryable<U>>`, which for each element of type `T` produces a series of `U`'s
        - then we can obtain an `IQueryable<U>`, which is a series of `U`'s, by concatenating all transformations of the original query
        - this is called `SelectMany`
        - examples of `SelectMany`        
- LINQ also comes equipped with a special sql-like syntax for creating "simple" queries
- this syntax makes it easier to write queries without having to work with higher order functions and such
- it yields very readable code, especially when joining or performing sub-queries
- the starting point is one or more `from V in Q`, each of which gives name `V` to all elements of `Q` (`Q : IQueryable<T>`)
- then we can apply a series of predicates by `where P`, where `P` is a boolean expression that depends on the `V`'s
- we can even perform sub-queries or declare new variables by using `let V = E`, where `E` is an arbitrary expression (even a full blown query!) and `V` is a fresh variable name
- when we are done, we can yield a result by `select E`, where `E` is an arbitrary expression
- so `dbContext.Student.Where(s => s.Age > 20).Select(s => s.Name)` would become the arguably more readable:
```
from s in dbContext.Student
where s.Age > 20
select s.Name
```
- we can also perform sub queries such as
```
from c in dbContext.Class
let students = from s in dbContext.Student
               where s.ClassId == c.Id
               select s
select Tuple.Create(c, students)
```
- show how the queries above are translated with `SelectMany`
- show some more examples of queries, for example all lectures from all courses that a given student `s` may see
- and much, much, more (again, GIYF)

## Extra materials for interested students (and teachers)

We can also point out that LINQ is a much more general tool than just a way to write LINQ queries.

- let us build a generic container, `M<T>` (for example `Option<T>`)
- let us add two static extension methods:
    - `ToM : T => M<T>`
    - `SelectMany : M<T>, Func<T, M<U>>, Func<T, U, V> => M<V>`
- at this point we can use the LINQ syntactic sugar on objects of type `M<T>` instead `IQueryable<T>`:
```
var v =
  from x in (20.ToOption())
  from y in (4.ToOption())
  select x / y;
```
- to see a full implementation of this, head over to [github](https://github.com/hogeschool/Development-5/tree/2017-2018/Lectures/Lecture2/CodeSamples/OptionMonad) 
- as long as `M` offers an implementation for such methods, then we say that `M` is a _monad_ and LINQ will support it with syntactic sugar
    - examples of monads range from multithreading, to reactive programming, to coroutines, and much much more
    - monads are one of the most powerful and flexible tools to perform meta-programming, and are an incredibly flexible design pattern